# Environment Setup

To interact with this notebook, you should first clone our [GitHub repo](https://github.com/choH/lottery_regulated_grouped_kernel_pruning) into the folder you setup for syncing with Google Drive (or simply upload our repo's folder to your Google Drive). By default, this syncing folder should be `My Drive` — if this is not the case, you need to change the `cd` line.

(and yes, we are using an ancient version of `sk-learn` as we utilize [`ndanielsen/Same-Size-K-Means`](https://github.com/ndanielsen/Same-Size-K-Means) for equal-size $k$-means.)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip install numpy==1.19
!pip install scikit-learn==0.20.1

!nvidia-smi

In [2]:
%cd /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning

/content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning


# Demo: Complete Task Walkthrough on ResNet-20

To demo the usage of this codebase, let's do a complete walkthrough on a CIFAR ResNet-20. We'd first initialize and *save* an untrained model: `baseline/resnet20_cifar10_init.pt`.

In [17]:
import torch
import models.resnet

init_model_save_dir = '/content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet20_cifar10_init.pt'
model = models.resnet.resnet20()
print(model)

torch.save(model, init_model_save_dir)

CifarResNet : Depth : 20 , Layers for each block : 3
CifarResNet(
  (conv_1_3x3): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (stage_1): Sequential(
    (0): ResNetBasicblock(
      (conv_a): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn_a): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv_b): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn_b): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResNetBasicblock(
      (conv_a): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn_a): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv_b): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (b

With the untrained model saved, we'd supply it to the `main.py` with a setting file of `resnet_train.json` and `--task` set to `train`. This combination will train the model at `--model_dir` according to the setting specified in `resnet_train.json`; which has the correct training setting used for reported experiments in our paper, yet saving its snapshots along the way.

In [18]:
!python main.py \
--exp_desc resnet20_cifar10_baseline_train \
--setting_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet_train.json \
--model_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet20_cifar10_init.pt \
--output_folder_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/output/resnet20_cifar10_baseline_train \
--baseline True \
--task train

2022-11-24 00:17:44,884 | INFO : Experiment resnet20_cifar10_baseline_train starts at 2022-11-24T00:17:44.880115.
2022-11-24 00:17:44,885 | INFO : Parsed setting file from /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet_train.json...
2022-11-24 00:17:44,912 | INFO : ==> Preparing dataset: cifar10
Files already downloaded and verified
Files already downloaded and verified
2022-11-24 00:17:47,218 | INFO : ==> Building baseline model: /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet20_cifar10_init.pt (dataset: cifar10)
2022-11-24 00:17:48,115 | INFO : Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-11-24 00:17:48,115 | INFO : NumExpr defaulting to 8 threads.
2022-11-24 00:17:49,978 | INFO : Model /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet20_cifar10_init.pt loaded (<class 'models.resnet.CifarResNet'>)
2022-11-24 00:17:53,514 | INFO : Mode

With a trained baseline model and its snapshots obtained, we may now prune and finetune the trained model. Note in this case, we set `--task` to `finetune` and supply the `main.py` with a setting file that includes pruning/fine-tuning specifications (`resnet20_cifar10_prune_and_finetune.json`) and a folder dir that includes all the corresponding snapshot files (at `--snapsho_folder_dir`).

In [20]:
!python main.py \
--exp_desc resnet20_cifar10_demo_exp \
--setting_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet20_cifar10_prune_and_finetune.json \
--model_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/output/resnet20_cifar10_train/trained \
--snapshot_folder_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/output/resnet20_cifar10_baseline_train/checkpoints \
--output_folder_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/output/resnet20_cifar10_demo_exp \
--baseline True \
--task finetune

2022-11-24 01:33:00,250 | INFO : Experiment resnet20_cifar10_demo_exp starts at 2022-11-24T01:33:00.246175.
2022-11-24 01:33:00,250 | INFO : Parsed setting file from /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet20_cifar10_prune_and_finetune.json...
2022-11-24 01:33:00,278 | INFO : ==> Preparing dataset: cifar10
Files already downloaded and verified
Files already downloaded and verified
2022-11-24 01:33:02,597 | INFO : ==> Building baseline model: /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/output/resnet20_cifar10_train/trained (dataset: cifar10)
2022-11-24 01:33:03,483 | INFO : Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-11-24 01:33:03,483 | INFO : NumExpr defaulting to 8 threads.
2022-11-24 01:33:05,339 | INFO : Model /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/output/resnet20_cifar10_train/trained loaded (<class 'models.resnet.CifarResNet'>)
2022-11-24 0

If you are here, you have now completed a full task cycle of TMI-GKP. Congratulations — we thank you for your attention.

---

To give a relatively deeper dive of our code base, the basic logic is the `main.py` file will take a baseline model, the baseline model's training snapshot folder for TMI evaluation, and a pruning setting file. You will also need to specify an output folder, where upon completion of this `--task finetune` experiment, you should find:

* `baseline`: Copy of the baseline model
* `pruned`: The pruned model (with clustering information to extract a `cluster.json` as described in Section A.3.2 of the paper).
* `grouped`: The grouped model. This is basically `pruned` but in grouped convolution architecture (with sparsity removed). All non-inference-related portions are also removed (e.g., the clustering info).
* `finetuned`: The finetuned model trained upon `grouped`.
* `setting.json`: Copy of the assigned setting file, but with more information stored (e.g., configuration assigned via `argparse`, log of acc/loss, etc.). Should you want to rerun this experiment, you should pass the original setting file (in this case, `settings/resnet20_cifar10_prune_and_finetune.json`) but not this one.
* `experiment.log`: Experiment printouts registered by the logger. This will also print to the terminal for easier monitoring.
* `/checkpoints/` folder: Saved checkpoints during the finetuning process.

---

Please do not hesitate to open an issue for questions or discussions. Being my undergraduate work, I'd be first to admit — and there's no denying with the complete lack of docs and comments — this is not the cleanest codebase you'd work with, but I keep a close monitor of this repo, yet I put in serious efforts to help people. In fact, this demo update is inspired by the discussion with [@DavidYanAnDe](https://github.com/DavidYanAnDe) in [Issue #2](https://github.com/choH/lottery_regulated_grouped_kernel_pruning/issues/2), I'd thank him for help me improve the reproducibility and usability of the codebase.

**We the authors believe *Grouped Kernel Pruning* is a promising direction for future structured pruning methods, as it makes sense to pursue a higher degree of pruning freedom yet retain those densely structured properties** (empirical results also suggest a one-shot adoption of GKP is often better than most iterative filter pruning methods with much higher retraining/finetuning budget). We'd hope more people from the pruning community may explore it in lieu of the already crowded filter/channel pruning realms.

Good day :)

# Experiment Showcase on Reported ResNet-32/56/110

The following three experiments are ResNet-32, ResNet-56, and ResNet-110 on CIFAR-10. You are able to replicate the exact settings of these experiment by using their respective setting file. However, you will have to obtain a trained baseline model with training snapshots first (they are too large to store in the repo). These are provided here for reference/demonstration purposes.

The ResNet-32 and ResNet-56 experiments below are actually the reported ones in Table 2 of our paper. For ResNet-110, Colab unfortunately stalled on us so the experiment was interapted. Since we are too lazy to "baby" another run on Colab, we have supplemented the `experiment.log` and setting file of the reported experiment (`94.26 pruned + 0.64 delta = 94.90 finetuned acc.`) should you want to checkout or replicate.

In [ ]:
!python main.py \
--exp_desc resnet32_cifar10_demo  \
--setting_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet32_cifar10_reported.json \
--model_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet32_cifar10_baseline \
--snapshot_folder_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/snapshots/resnet32/ \
--output_folder_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/output/resnet32_cifar10_demo_run \
--baseline True \
--task finetune

2022-03-11 04:35:35,895 | INFO : Experiment resnet32_cifar10_demo starts at 2022-03-11T04:35:35.891115.
2022-03-11 04:35:35,895 | INFO : Parsed setting file from /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet32_cifar10_reported.json...
2022-03-11 04:35:35,910 | INFO : ==> Preparing dataset: cifar10
Files already downloaded and verified
Files already downloaded and verified
2022-03-11 04:35:38,134 | INFO : ==> Building baseline model: /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet32_cifar10_baseline (dataset: cifar10)
2022-03-11 04:35:41,554 | INFO : Model /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet32_cifar10_baseline loaded (<class 'models.resnet.CifarResNet'>)
2022-03-11 04:35:42,556 | INFO : Model /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet32_cifar10_baseline <class 'models.resnet.CifarResNet'> has a baseline of 92.82.
2022-03-11 04:35:42,570 | INF

In [ ]:
!python main.py \
--exp_desc resnet56_cifar10_demo  \
--setting_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet56_cifar10_reported.json \
--model_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet56_cifar10_baseline \
--snapshot_folder_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/snapshots/resnet56/ \
--output_folder_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/output/resnet56_cifar10_demo_run \
--baseline True \
--task finetune

2022-03-11 10:16:08,058 | INFO : Experiment resnet56_cifar10_demo starts at 2022-03-11T10:16:08.054065.
2022-03-11 10:16:08,058 | INFO : Parsed setting file from /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet56_cifar10_reported.json...
2022-03-11 10:16:08,074 | INFO : ==> Preparing dataset: cifar10
Files already downloaded and verified
Files already downloaded and verified
2022-03-11 10:16:10,366 | INFO : ==> Building baseline model: /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet56_cifar10_baseline (dataset: cifar10)
2022-03-11 10:16:15,250 | INFO : Model /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet56_cifar10_baseline loaded (<class 'models.resnet.CifarResNet'>)
2022-03-11 10:16:16,612 | INFO : Model /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet56_cifar10_baseline <class 'models.resnet.CifarResNet'> has a baseline of 93.78.
2022-03-11 10:16:16,635 | INF

In [ ]:
!python main.py \
--exp_desc resnet110_cifar10_demo  \
--setting_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet110_cifar10_reported.json \
--model_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet110_cifar10_baseline \
--snapshot_folder_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/snapshots/resnet110/ \
--output_folder_dir /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/output/resnet110_cifar10_demo \
--baseline True \
--task finetune

2022-03-14 03:14:04,895 | INFO : Experiment resnet110_cifar10_demo starts at 2022-03-14T03:14:04.136283.
2022-03-14 03:14:04,895 | INFO : Parsed setting file from /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/settings/resnet110_cifar10_reported.json...
2022-03-14 03:14:04,924 | INFO : ==> Preparing dataset: cifar10
Files already downloaded and verified
Files already downloaded and verified
2022-03-14 03:14:07,204 | INFO : ==> Building baseline model: /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet110_cifar10_baseline (dataset: cifar10)
2022-03-14 03:14:11,873 | INFO : Model /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet110_cifar10_baseline loaded (<class 'models.resnet.CifarResNet'>)
2022-03-14 03:14:14,962 | INFO : Model /content/drive/MyDrive/lottery_regulated_grouped_kernel_pruning/baseline/resnet110_cifar10_baseline <class 'models.resnet.CifarResNet'> has a baseline of 94.26.
2022-03-14 03:14:15,012 